In [94]:
from graphframes import GraphFrame

In [ ]:
#%run ./etl_trusted_features.ipynb
#df = readFirstEvents()

In [2]:
%run ./read_file.ipynb
df = spark.read.parquet(f"{s3_url_trusted}/prod/graph/first_events")\
            .withColumn("image_path", getFileUDF(col("image_path"))) \
            .withColumn("parent_image_path", getFileUDF(col("parent_image_path"))) \
            .withColumn("new_path", getFileUDF(col("new_path"))) \
            .withColumn("file_path", getFileUDF(col("file_path"))) \
            .cache()

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-58076c5b-3452-4736-ae91-032e376b6d5c;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 140ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/19 21:54:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 21:54:07 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/19 21:54:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/19 21:54:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/03/19 21:54:08 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
df_mal = df.where(col('malicious') == 1)
print("mal count: " +str(df_mal.count()))

mal count: 3642


In [95]:
#down select to match time windown between benign and malicious
df_b = df.where(col('malicious') == 0)
mal_hosts = list(df.select('hostname').filter( col('malicious') == 1 ).distinct().toPandas()['hostname'])
df_b = df_b.filter(col("hostname").isin(mal_hosts)).cache()
df_b_ds = df_b.where((col('event_hour') == 16) & (col('event_minute') > 29)).cache()
df_b_ds_4 = df_b_ds.limit(4000) 
df_b_ds.count()

72711

In [96]:
df_b_ds2 = df_b.where((col('event_hour') == 16) & (col('event_minute') < 30)).cache()
df_b_ds_4b = df_b_ds2.limit(4000) 

In [97]:
# Create distinct vertices with source as actorid, destination as objectid for bengin
src_vertices = df_b_ds_4.selectExpr('actorID as id').distinct()
dst_vertices = df_b_ds_4.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_b_ds_4.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

#create paths and count
# filter paths to only those where all edges are connected
graph_b_4 = motifs6.filter("e1.dst = e2.src and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src").cache()
print("event traces: "+str(graph_b_4.count()))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


event traces: 1882


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [98]:
# Create distinct vertices with source as actorid, destination as objectid for bengin 10k
src_vertices = df_b_ds_4b.selectExpr('actorID as id').distinct()
dst_vertices = df_b_ds_4b.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_b_ds_4b.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")

#create paths and count
# filter paths to only those where all edges are connected
graph_b_4b = motifs6.filter("e1.dst = e2.src and e2.dst = e3.src and e3.dst = e4.src and e4.dst = e5.src and e5.dst = e6.src").cache()
print("event traces: "+str(graph_b_4b.count()))

event traces: 1500


In [99]:
%run trace_encode.ipynb

In [9]:
graph_mal = create_graph(df_mal)

event traces: 21135


In [56]:
graph_b_5b = create_graph(df_mal)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


event traces: 21135


In [58]:
graph_b_4b.limit(1).toPandas()

,a,e1,b,e2,c,e3,d,e4,e,e5,f,e6,g
0,"(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)"


In [100]:
#create dataframes for processing 
graph_b = graph_b_4b.union(graph_b_4)
graph_b.count()

3382

In [59]:
graph_b.limit(1).toPandas()

,a,e1,b,e2,c,e3,d,e4,e,e5,f,e6,g
0,"(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)","(063fc5a9-02d0-4b48-8738-d4af953d55cd, 063fc5a...","(063fc5a9-02d0-4b48-8738-d4af953d55cd,)"


In [101]:
#randomly select event traces from graph_mal
graph_mal_sel = graph_mal.distinct().orderBy(rand()).limit(3382)

In [12]:
#check count
graph_mal_sel.count()

3382

In [102]:
all_events = graph_mal_sel.union(graph_b)

In [103]:
df_total, map_total = trace_encode(all_events, ['object','action',
             'image_path','parent_image_path'], output = 'vec')

transposed explode: 0.5826127529144287


one-hot time: 11.336900472640991


total elapsed time: 14.083482027053833


In [104]:
map_total['image_path']

['powershell.exe',
 'cmd.exe',
 'conhost.exe',
 'PING.EXE',
 'taskkill.exe',
 'SearchProtocolHost.exe',
 'DllHost.exe',
 'GoogleUpdate.exe',
 'firefox.exe',
 'svchost.exe',
 'geckodriver.exe',
 'python.EXE',
 'python.exe',
 'reg.exe',
 'OneDriveSetup.exe',
 'schtasks.exe',
 'csrss.exe',
 'services.exe',
 'taskhostw.exe',
 'EXCEL.EXE',
 'sppsvc.exe',
 'TiWorker.exe',
 'TrustedInstaller.exe',
 'WinStore.App.exe',
 'winlogon.exe',
 'sc.exe',
 'AcroRd32.exe',
 'Explorer.EXE',
 'gpupdate.exe',
 'msoia.exe',
 'SearchFilterHost.exe',
 'bcdedit.exe',
 'InstallAgent.exe',
 'mantra.exe',
 'net.exe',
 'csc.exe',
 'fontdrvhost.exe']

In [105]:
map_total['parent_image_path']

['powershell.exe',
 'cmd.exe',
 'conhost.exe',
 'PING.EXE',
 'wmiprvse.exe',
 'taskkill.exe',
 'SearchProtocolHost.exe',
 'DllHost.exe',
 'GoogleUpdate.exe',
 'svchost.exe',
 'firefox.exe',
 'geckodriver.exe',
 'python.exe',
 'OneDriveSetup.exe',
 'csrss.exe',
 'schtasks.exe',
 'python.EXE',
 'services.exe',
 'sppsvc.exe',
 'EXCEL.EXE',
 'taskhostw.exe',
 'TiWorker.exe',
 'TrustedInstaller.exe',
 'winlogon.exe',
 'WinStore.App.exe',
 'reg.exe',
 'sc.exe',
 'Explorer.EXE',
 'AcroRd32.exe',
 'msoia.exe',
 'SearchFilterHost.exe',
 'bcdedit.exe',
 'InstallAgent.exe',
 'gpupdate.exe',
 'mantra.exe',
 'SearchIndexer.exe']

In [78]:
print(df_total.count())
print(df_total.where(col('malicious') == 1).count())
print(df_total.where(col('malicious') == 0).count())

6764


3382


3382


In [79]:
df_total.where(col('malicious') == 0).limit(1).toPandas()

,Trace,malicious,event_sequence
0,8589934593,0,[100001000000000001000000000000000000000000000...


In [80]:
df_total.where(col('malicious') == 1).limit(1).toPandas()

,Trace,malicious,event_sequence
0,1,1,[100010000000001000000000000000000000000000000...


In [83]:
len(df_total.where(col('malicious') == 0).limit(1).toPandas()['event_sequence'][0][0])

91

In [84]:
len(df_total.where(col('malicious') == 1).limit(1).toPandas()['event_sequence'][0][0])

91

In [61]:
#process everything
te_m = trace_encode(graph_mal_sel, ['object','action',
             'image_path','parent_image_path'], output = 'vec')
te_m[0].limit(1).toPandas()['event_sequence']

transposed explode: 0.2352433204650879


one-hot time: 6.5506391525268555


total elapsed time: 8.141122102737427


0    [100010000000100000001000, 1000100000001000000...
Name: event_sequence, dtype: object

In [85]:
print(len(te_m[0].limit(1).toPandas()['event_sequence'][0][0]))
print(len(te_b[0].limit(1).toPandas()['event_sequence'][0][0]))

24


82


In [62]:
#process everything
te_b = trace_encode(graph_b, ['object','action',
             'image_path','parent_image_path'], output = 'vec')
te_b[0].limit(1).toPandas()['event_sequence']

transposed explode: 0.43525028228759766
one-hot time: 4.052865505218506
total elapsed time: 5.023257493972778


0    [100010000000001000000000000000000000000000000...
Name: event_sequence, dtype: object

In [63]:
te_b2 = trace_encode(graph_b_4b, ['object','action',
             'image_path','parent_image_path'], output = 'vec')
te_b2[0].limit(1).toPandas()['event_sequence']

transposed explode: 0.2649877071380615
one-hot time: 1.5210518836975098
total elapsed time: 1.8901464939117432


0    [100010000001000000000000000000000000001000000...
Name: event_sequence, dtype: object

In [50]:
print(te_b[0].count())
te_b[0].limit(2).toPandas()

3382


,Trace,malicious,event_sequence
0,1,0,[100010000000001000000000000000000000000000000...
1,2,0,[100100000000010000000000000000000000000000000...


In [51]:
print(te_m[0].count())
te_m[0].limit(2).toPandas()

3382


,Trace,malicious,event_sequence
0,1,1,"[100010000000100000001000, 1000100000001000000..."
1,2,1,"[100001000000100000000100, 1000100000001000000..."


In [64]:
#first combine both te dfs
df_all = te_b[0].unionByName(te_m[0])
print(df_all.count())
print(df_all.where(col('malicious') == 1).count())
print(df_all.where(col('malicious') == 0).count())
df_all.limit(2).toPandas()

6764


3382


3382


,Trace,malicious,event_sequence
0,1,0,[100010000000001000000000000000000000000000000...
1,2,0,[100100000000010000000000000000000000000000000...


In [30]:
#first expand vector list into one string (TODO: do this earlier)
str_create_udf = udf(lambda l: ' '.join(l), StringType())
# Convert the "event_sequence" column into a string
df_all = df_all.withColumn("event_sequence_str", str_create_udf(df_all["event_sequence"]))
#df_all = df_all.drop('event_sequence')

In [27]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import CountVectorizer

In [86]:
# Use CountVectorizer to convert the "event_sequence_str" column into a feature vector
cv = CountVectorizer(inputCol="event_sequence", outputCol="features")
model = cv.fit(df_total)
df_all = model.transform(df_total)

In [87]:
#just kidding turns out you need a string array, onto the train test split but first let me take a peek
df_all.limit(2).show() 

+-----+---------+--------------------+--------------------+
|Trace|malicious|      event_sequence|            features|
+-----+---------+--------------------+--------------------+
|    1|        1|[1000100000000010...|(99,[0,4,5],[4.0,...|
|    2|        1|[1000001000000010...|(99,[0,4,5],[4.0,...|
+-----+---------+--------------------+--------------------+



In [69]:
print(df_all.where(col('malicious') == 1).count())
print(df_all.where(col('malicious') == 0).count())

3382


3382


In [88]:
#split everything for train test split then re-attach
df_m = df_all.where(col('malicious') == 1)
df_b = df_all.where(col('malicious') == 0)
(train_m, test_m) = df_m.randomSplit([0.7, 0.3])
(train_b, test_b) = df_b.randomSplit([0.7, 0.3])
train_all = train_m.union(train_b)
test_all = test_m.union(test_b)




In [89]:
retrain, retest = df_all.randomSplit([0.7, 0.3])

In [90]:
#train the model
nb = NaiveBayes(featuresCol="features", labelCol="malicious")
nbModel = nb.fit(train_all)

In [91]:
#develop predictions using model
pred = nbModel.transform(test_all)

In [73]:
pred.limit(5).toPandas()

,Trace,malicious,event_sequence,features,rawPrediction,probability,prediction
0,3,0,[100010000000100000000000000000000000000000000...,"(0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-9.25832875179826, -58.11690855625178]","[1.0, 6.039324943717724e-22]",0.0
1,9,0,[100100000000000000100000000000000000000000000...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-29.265416651545117, -58.11690855625177]","[0.9999999999997049, 2.9509120858695996e-13]",0.0
2,12,0,[100100000000000000100000000000000000000000000...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-29.265416651545117, -58.11690855625177]","[0.9999999999997049, 2.9509120858695996e-13]",0.0
3,15,0,[100100000000000000100000000000000000000000000...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-30.151170659559188, -58.11690855625177]","[0.9999999999992844, 7.155407125507849e-13]",0.0
4,16,0,[100100000000000000100000000000000000000000000...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-26.14383747432672, -58.11690855625177]","[0.9999999999999869, 1.3009831137296168e-14]",0.0


In [92]:
# Evaluate the performance of the classifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
for metric in ["accuracy", "weightedPrecision","weightedRecall"]: 
    evaluator = MulticlassClassificationEvaluator(labelCol="malicious", predictionCol="prediction",
                                                  metricName=metric)
    results = evaluator.evaluate(pred)
    print("Test set " + metric+" = " + str(results))

Test set accuracy = 0.9975550122249389


Test set weightedPrecision = 0.9975666550238678


Test set weightedRecall = 0.9975550122249388


In [93]:
#train the model
nb = NaiveBayes(featuresCol="features", labelCol="malicious")
nbModel = nb.fit(retrain)
#develop predictions using model
repred = nbModel.transform(retest)
# Evaluate the performance of the classifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
for metric in ["accuracy", "weightedPrecision","weightedRecall"]: 
    evaluator = MulticlassClassificationEvaluator(labelCol="malicious", predictionCol="prediction",
                                                  metricName=metric)
    results = evaluator.evaluate(repred)
    print("Re-test set " + metric+" = " + str(results))

Re-test set accuracy = 0.9980879541108987


Re-test set weightedPrecision = 0.9980951830178517


Re-test set weightedRecall = 0.9980879541108987


## run reference model with new data. first read in the data


In [1]:
%run ./read_file.ipynb
s3_url_trusted = "s3a://sapient-bucket-trusted/"
trace_23_3 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/encoded/real/23Sep3").drop('malicious').cache()

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-57cc1c93-f00a-40d3-b237-b9aed874e309;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (41ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (28ms)
:: resolution report :: resolve 587ms :: artifacts dl 71ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/31 23:33:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/31 23:33:18 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
trace_23_3.printSchema()

root
 |-- Trace: long (nullable = true)
 |-- mal_trace: integer (nullable = true)
 |-- event_sequence: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [4]:
trace_23_3.count()

17373568

In [5]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
# Use CountVectorizer to convert the "event_sequence_str" column into a feature vector
cv = CountVectorizer(inputCol="event_sequence", outputCol="features")
model = cv.fit(trace_23_3)
trace_23_3 = model.transform(trace_23_3).cache()

In [8]:
trace_23_3.limit(3).show()

+-----+---------+--------------------+--------------------+
|Trace|mal_trace|      event_sequence|            features|
+-----+---------+--------------------+--------------------+
|   41|        0|[0100000000010000...|(111,[3,5,12],[1....|
|  499|        0|[0001000000010000...|(111,[2,8,12],[1....|
|  907|        0|[0100000000001000...|(111,[1,3,15],[1....|
+-----+---------+--------------------+--------------------+



In [9]:
print(trace_23_3.where(col('mal_trace')==1).count())
print(trace_23_3.where(col('mal_trace')==0).count())

118763
17254805


In [10]:
#split everything for train test split then re-attach
df_m = trace_23_3.where(col('mal_trace') == 1)
df_b = trace_23_3.where(col('mal_trace') == 0)
(train_m, test_m) = df_m.randomSplit([0.7, 0.3])
(train_b, test_b) = df_b.randomSplit([0.7, 0.3])
train_all = train_m.union(train_b)
test_all = test_m.union(test_b)

In [13]:
#train the model
nb = NaiveBayes(featuresCol="features", labelCol="mal_trace")
nbModel = nb.fit(train_all)
pred = nbModel.transform(test_all)

In [16]:
for metric in ["accuracy", "weightedPrecision","weightedRecall"]: 
    evaluator = MulticlassClassificationEvaluator(labelCol="mal_trace", predictionCol="prediction",
                                                  metricName=metric)
    results = evaluator.evaluate(pred)
    print("Test set " + metric+" = " + str(results))

Test set accuracy = 0.9932324467306498


Test set weightedPrecision = 0.9916294182749649


Test set weightedRecall = 0.9932324467306498


In [18]:
pred.limit(2).show()

+------+---------+--------------------+--------------------+--------------------+--------------------+----------+
| Trace|mal_trace|      event_sequence|            features|       rawPrediction|         probability|prediction|
+------+---------+--------------------+--------------------+--------------------+--------------------+----------+
|132489|        1|[1000000000001000...|(111,[0,4,34],[1....|[-10.508846329598...|[0.96975449670258...|       0.0|
|132497|        1|[1000000000001000...|(111,[0,4,34],[1....|[-10.508846329598...|[0.96975449670258...|       0.0|
+------+---------+--------------------+--------------------+--------------------+--------------------+----------+



In [20]:
y_true = pred.select(['mal_trace']).collect()
y_pred = pred.select(['prediction']).collect()

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00   5177201
           1       0.76      0.00      0.00     35318

    accuracy                           0.99   5212519
   macro avg       0.87      0.50      0.50   5212519
weighted avg       0.99      0.99      0.99   5212519



In [24]:
predictionsAndLabels = pred.select("prediction", "mal_trace")
confusionMatrix = predictionsAndLabels.rdd.map(lambda x: (x[0], x[1])).countByValue()
print("Confusion matrix:")
print("pred-actual: count")
for ((predicted, actual), count) in confusionMatrix.items():
    print("(%d, %d): %d" % (predicted, actual, count))

Confusion matrix:
pred-actual: count
(0, 1): 35256
(1, 1): 62
(0, 0): 5177181
(1, 0): 20


In [3]:
#rerun with even data set
#first randomly select benign traces
df_b_trunc = trace_23_3.where(col('mal_trace')==0)
df_b_trunc = df_b_trunc.distinct().orderBy(rand()).limit(118763)
df_m = trace_23_3.where(col('mal_trace')==1)
#combine datasets to re-run tokenizer
df_all_even = df_m.union(df_b_trunc)

In [31]:
df_all_even = df_all_even.drop('features')

In [6]:
#rerun everything
# Use CountVectorizer to convert the "event_sequence_str" column into a feature vector
cv = CountVectorizer(inputCol="event_sequence", outputCol="features")
model = cv.fit(df_all_even)
df_all_even = model.transform(df_all_even).cache()

In [7]:
#split everything for train test split then re-attach
df_m = df_all_even.where(col('mal_trace') == 1)
df_b = df_all_even.where(col('mal_trace') == 0)
(train_m, test_m) = df_m.randomSplit([0.7, 0.3])
(train_b, test_b) = df_b.randomSplit([0.7, 0.3])

In [ ]:
train_all = train_m.union(train_b)
test_all = test_m.union(test_b)
#train the model
nb = NaiveBayes(featuresCol="features", labelCol="mal_trace")
nbModel = nb.fit(train_all)
pred = nbModel.transform(test_all)

In [34]:
y_true = pred.select(['mal_trace']).collect()
y_pred = pred.select(['prediction']).collect()

print(classification_report(y_true, y_pred))

predictionsAndLabels = pred.select("prediction", "mal_trace")
confusionMatrix = predictionsAndLabels.rdd.map(lambda x: (x[0], x[1])).countByValue()
print("Confusion matrix:")
print("pred-actual: count")
for ((predicted, actual), count) in confusionMatrix.items():
    print("(%d, %d): %d" % (predicted, actual, count))

              precision    recall  f1-score   support

           0       0.75      0.69      0.72     35580
           1       0.72      0.77      0.74     35824

    accuracy                           0.73     71404
   macro avg       0.73      0.73      0.73     71404
weighted avg       0.73      0.73      0.73     71404

Confusion matrix:
pred-actual: count
(0, 1): 8141
(1, 1): 27683
(0, 0): 24632
(1, 0): 10948


In [11]:
#35849 count for test benign
test_m_r = test_m.distinct().orderBy(rand()).limit(359)

In [12]:
#train and test with a 99% reduction in malicious test data
train_all = train_m.union(train_b)
test_all = test_m_r.union(test_b)
#train the model
nb = NaiveBayes(featuresCol="features", labelCol="mal_trace")
nbModel = nb.fit(train_all)
pred = nbModel.transform(test_all)

In [13]:
y_true = pred.select(['mal_trace']).collect()
y_pred = pred.select(['prediction']).collect()

print(classification_report(y_true, y_pred))

predictionsAndLabels = pred.select("prediction", "mal_trace")
confusionMatrix = predictionsAndLabels.rdd.map(lambda x: (x[0], x[1])).countByValue()
print("Confusion matrix:")
print("pred-actual: count")
for ((predicted, actual), count) in confusionMatrix.items():
    print("(%d, %d): %d" % (predicted, actual, count))

              precision    recall  f1-score   support

           0       1.00      0.69      0.82     35849
           1       0.02      0.74      0.05       359

    accuracy                           0.69     36208
   macro avg       0.51      0.72      0.43     36208
weighted avg       0.99      0.69      0.81     36208



Confusion matrix:
pred-actual: count
(0, 1): 94
(1, 1): 265
(0, 0): 24892
(1, 0): 10957


### run model based on expanded image path columns

In [14]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
trace_23_3_p = spark.read.parquet(f"{s3_url_trusted}/prod/graph/encoded/real/23Sep3_paths").drop('malicious').cache()

In [16]:
#rerun with even data set
#first randomly select benign traces
df_b_trunc = trace_23_3_p.where(col('mal_trace')==0)
df_b_trunc = df_b_trunc.distinct().orderBy(rand()).limit(118763)
df_m = trace_23_3_p.where(col('mal_trace')==1)
#combine datasets to re-run tokenizer
df_all_even = df_m.union(df_b_trunc)

In [17]:
#rerun everything
# Use CountVectorizer to convert the "event_sequence_str" column into a feature vector
cv = CountVectorizer(inputCol="event_sequence", outputCol="features")
model = cv.fit(df_all_even)
df_all_even = model.transform(df_all_even).cache()

In [18]:
#split everything for train test split then re-attach
df_m = df_all_even.where(col('mal_trace') == 1)
df_b = df_all_even.where(col('mal_trace') == 0)
(train_m, test_m) = df_m.randomSplit([0.7, 0.3])
(train_b, test_b) = df_b.randomSplit([0.7, 0.3])

In [19]:
train_all = train_m.union(train_b)
test_all = test_m.union(test_b)
#train the model
nb = NaiveBayes(featuresCol="features", labelCol="mal_trace")
nbModel = nb.fit(train_all)
pred = nbModel.transform(test_all)

In [20]:
y_true = pred.select(['mal_trace']).collect()
y_pred = pred.select(['prediction']).collect()

print(classification_report(y_true, y_pred))

predictionsAndLabels = pred.select("prediction", "mal_trace")
confusionMatrix = predictionsAndLabels.rdd.map(lambda x: (x[0], x[1])).countByValue()
print("Confusion matrix:")
print("pred-actual: count")
for ((predicted, actual), count) in confusionMatrix.items():
    print("(%d, %d): %d" % (predicted, actual, count))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82     35752
           1       0.81      0.84      0.83     35561

    accuracy                           0.82     71313
   macro avg       0.82      0.82      0.82     71313
weighted avg       0.82      0.82      0.82     71313

Confusion matrix:
pred-actual: count
(0, 1): 5655
(1, 1): 29906
(0, 0): 28877
(1, 0): 6875


In [11]:
#NB function
def ref_NB(df):
    #rerun with even data set
    #first randomly select benign traces
    df_b_trunc = df.where(col('mal_trace')==0).cache()
    print("benign traces: " + str(df_b_trunc.count()))
    df_m = df.where(col('mal_trace')==1).cache()
    trunc = df_m.count()
    print("malicious traces: " + str(trunc))
    df_b_trunc = df_b_trunc.distinct().orderBy(rand()).limit(trunc).cache()
    #combine datasets to re-run tokenizer
    df_all_even = df_m.union(df_b_trunc).cache()
    # Use CountVectorizer to convert the "event_sequence_str" column into a feature vector
    cv = CountVectorizer(inputCol="event_sequence", outputCol="features")
    model = cv.fit(df_all_even)
    df_all_even = model.transform(df_all_even).cache()
    #split everything for train test split then re-attach
    df_m = df_all_even.where(col('mal_trace') == 1).cache()
    df_b = df_all_even.where(col('mal_trace') == 0).cache()
    (train_m, test_m) = df_m.randomSplit([0.7, 0.3])
    (train_b, test_b) = df_b.randomSplit([0.7, 0.3])
    train_all = train_m.union(train_b).cache()
    test_all = test_m.union(test_b).cache()
    #train the model
    nb = NaiveBayes(featuresCol="features", labelCol="mal_trace")
    nbModel = nb.fit(train_all)
    pred = nbModel.transform(test_all).cache()
    y_true = pred.select(['mal_trace']).collect()
    y_pred = pred.select(['prediction']).collect()

    print(classification_report(y_true, y_pred))

    predictionsAndLabels = pred.select("prediction", "mal_trace")
    confusionMatrix = predictionsAndLabels.rdd.map(lambda x: (x[0], x[1])).countByValue()
    print("Confusion matrix:")
    print("pred-actual: count")
    for ((predicted, actual), count) in confusionMatrix.items():
        print("(%d, %d): %d" % (predicted, actual, count))
    

### Now test on a full 6 event trace from Day 1

In [2]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-45095160-48dc-4902-be79-6eeb5a975499;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 178ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/01 14:08:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/01 14:08:04 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/04/01 14:08:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
trace_23_6 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/encoded/real/23Sep6").drop('malicious').cache()

In [6]:
#results with old truncation
ref_NB(trace_23_6)

              precision    recall  f1-score   support

           0       0.82      0.84      0.83     35587
           1       0.85      0.83      0.84     38686

    accuracy                           0.84     74273
   macro avg       0.84      0.84      0.84     74273
weighted avg       0.84      0.84      0.84     74273



Confusion matrix:
pred-actual: count
(0, 1): 6496
(1, 1): 32190
(0, 0): 29977
(1, 0): 5610


In [12]:
#results with new truncation
ref_NB(trace_23_6)

benign traces: 14256314
malicious traces: 128364


              precision    recall  f1-score   support

           0       0.81      0.85      0.83     38572
           1       0.84      0.80      0.82     38498

    accuracy                           0.82     77070
   macro avg       0.83      0.82      0.82     77070
weighted avg       0.83      0.82      0.82     77070

Confusion matrix:
pred-actual: count
(0, 1): 7591
(1, 1): 30907
(0, 0): 32654
(1, 0): 5918
